In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from statsmodels.stats.weightstats import ttest_ind
from matplotlib import pyplot as plt
from scipy import stats

In [2]:
nhanes = pd.read_csv('../data/nhanes.csv')
nhanes.head()

,id,heart_attack,relative_heart_attack,gender,age,race,edu,annual_income,weight,bmi,...,blood_press,blood_press2,hyper_med,hbq_med,high_chol,meadial_access,cover_hc,health_diet,year_smoke,year_hyper
0,93705,0,0,2,66,4,2,3,8614.571172,31.7,...,1,1,1,1,0,1,1,2,50,16
1,93708,0,0,2,66,5,1,6,13329.450589,23.7,...,1,1,1,1,1,1,1,4,0,16
2,93709,0,0,2,75,4,4,2,12043.388271,38.9,...,1,1,1,1,0,1,1,2,60,4
3,93711,0,1,1,56,5,5,15,11178.260106,21.3,...,0,0,0,0,1,1,1,3,0,0
4,93713,0,0,1,67,3,3,6,174806.575152,23.5,...,0,0,0,0,0,1,1,1,53,0


In [3]:
nhanes.columns

Index(['id', 'heart_attack', 'relative_heart_attack', 'gender', 'age', 'race',
       'edu', 'annual_income', 'weight', 'bmi', 'diabete', 'smoke_life',
       'phy_vigorous', 'phy_moderate', 'blood_press', 'blood_press2',
       'hyper_med', 'hbq_med', 'high_chol', 'meadial_access', 'cover_hc',
       'health_diet', 'year_smoke', 'year_hyper'],
      dtype='object')

In [4]:
nhanes_X = nhanes.drop(columns=['id', 'heart_attack','diabete','weight'])
nhanes_diab = nhanes['diabete']
weight = nhanes['weight']

## Estimate propensity score by fitting a logistic regression model.

In [5]:
lg = LogisticRegression(random_state=0, max_iter = 1000)
lg.fit(nhanes_X, nhanes_diab, sample_weight = weight)
prop_score = lg.predict_proba(nhanes_X)[:,1]

In [6]:
dia_idx = np.where(nhanes['diabete'].values==1)
non_dia_idx = np.where(nhanes['diabete'].values==0)

In [7]:
nhanes['diabete'][dia_idx[0]]

5       1
17      1
22      1
23      1
31      1
       ..
5237    1
5239    1
5249    1
5250    1
5257    1
Name: diabete, Length: 836, dtype: int64

## Use Nearest-Neighborhood to match the diabete and non-diabete patients on the estimated propensity scores.

In [8]:
prop_score_logit = np.log(prop_score / (1 - prop_score))
std = np.std(prop_score_logit[dia_idx])
std

1.162975281960774

In [9]:
result = [0]*len(prop_score_logit[dia_idx])
for i in range(len(prop_score_logit[dia_idx])):
    dif = prop_score_logit[dia_idx][i] - prop_score_logit[non_dia_idx]
    dif[np.array(result)[np.array(result)!=0]] = 100
    min_val = min(abs(dif))
    if min_val > 0.2*std:
        result[i] = 0
    else:
        result[i] = np.where(abs(dif)==min_val)[0][0]

In [10]:
np.sum(np.array(result) == 0)

23

In [11]:
result = np.array(result)
dia_idx_matched = dia_idx[0][result!=0]
result = result[result!=0]

In [12]:
matched_idx = non_dia_idx[0][result]
heart_matched = nhanes['heart_attack'].values[matched_idx]
heart_non_dia = nhanes['heart_attack'].values[non_dia_idx]
heart_dia_matched = nhanes['heart_attack'].values[dia_idx_matched]

## Use T-test to figure out the effect of diabetes on heart attack

In [15]:
# proportion of heart attack for people with diabetes
np.sum(heart_dia_matched)/len(heart_dia_matched)

0.13161131611316113

In [16]:
# proportion of heart attack for all the people don't have diabetese
np.sum(heart_non_dia)/len(heart_non_dia)

0.03183562881011515

In [17]:
# proportion of heart attack for people don't have diabetes matched by propensity score
np.sum(heart_matched)/len(heart_matched)

0.07626076260762607

In [19]:
stats.ttest_ind(heart_matched, heart_dia_matched)

Ttest_indResult(statistic=-3.6696579278558916, pvalue=0.00025072202075522923)

In [20]:
ttest_ind(heart_dia_matched, heart_matched, usevar='unequal', weights=(weight[dia_idx_matched], weight[matched_idx]))

(1213.7422353842449, 0.0, 40634737.39405689)

In [ ]:
heart_dia_matched

In [24]:
np.sum(dia_idx_matched)

2091154